In [1]:
import pandas as pd
import numpy as np
import os

# import dataframe

In [2]:
cust = pd.read_csv(r'/Users/r1121/Desktop/07-2023 Instacart Basket Analysis/02 Data/Original Data/customers.csv', index_col = False)

In [3]:
# check data
cust.head()

user_id First Name    Surnam  Gender       STATE  Age date_joined  \
0    26711    Deborah  Esquivel  Female    Missouri   48    1/1/2017   
1    33890   Patricia      Hart  Female  New Mexico   36    1/1/2017   
2    65803    Kenneth    Farley    Male       Idaho   35    1/1/2017   
3   125935   Michelle     Hicks  Female        Iowa   40    1/1/2017   
4   130797        Ann   Gilmore  Female    Maryland   26    1/1/2017   

   n_dependants fam_status  income  
0             3    married  165665  
1             0     single   59285  
2             2    married   99568  
3             0     single   42049  
4             1    married   40374

In [4]:
cust.shape

(206209, 10)

In [5]:
cust.describe()

user_id            Age   n_dependants         income
count  206209.000000  206209.000000  206209.000000  206209.000000
mean   103105.000000      49.501646       1.499823   94632.852548
std     59527.555167      18.480962       1.118433   42473.786988
min         1.000000      18.000000       0.000000   25903.000000
25%     51553.000000      33.000000       0.000000   59874.000000
50%    103105.000000      49.000000       1.000000   93547.000000
75%    154657.000000      66.000000       3.000000  124244.000000
max    206209.000000      81.000000       3.000000  593901.000000

In [6]:
cust['n_dependants'].value_counts(dropna = False)

0    51602
3    51594
1    51531
2    51482
Name: n_dependants, dtype: int64

In [7]:
cust['fam_status'].value_counts(dropna = False)

married                             144906
single                               33962
divorced/widowed                     17640
living with parents and siblings      9701
Name: fam_status, dtype: int64

# wrangle the data so that it follows consistent logic

In [8]:
# drop columns that do not contribute a lot to my analysis.
cust_dropcol = cust.drop(columns=['First Name', 'Surnam', 'date_joined'])

In [9]:
cust_dropcol.head()

user_id  Gender       STATE  Age  n_dependants fam_status  income
0    26711  Female    Missouri   48             3    married  165665
1    33890  Female  New Mexico   36             0     single   59285
2    65803    Male       Idaho   35             2    married   99568
3   125935  Female        Iowa   40             0     single   42049
4   130797  Female    Maryland   26             1    married   40374

In [10]:
# rename columns to lower case
cust_dropcol.rename(columns = {'Gender':'gender'}, inplace = True)
cust_dropcol.rename(columns = {'STATE':'state'}, inplace = True)
cust_dropcol.rename(columns = {'Age':'age'}, inplace = True)

In [11]:
cust_dropcol.head()

user_id  gender       state  age  n_dependants fam_status  income
0    26711  Female    Missouri   48             3    married  165665
1    33890  Female  New Mexico   36             0     single   59285
2    65803    Male       Idaho   35             2    married   99568
3   125935  Female        Iowa   40             0     single   42049
4   130797  Female    Maryland   26             1    married   40374

# conduct the fundamental data quality and consistency checks

In [12]:
# check for mixed-type data
for col in cust_dropcol.columns.tolist():
    weird = (cust_dropcol[[col]].applymap(type) != cust_dropcol[[col]].iloc[0].apply(type)).any(axis = 1)
    if len(cust_dropcol[weird]) > 0:
        print (col)
    else:
        print ('No mixed type')

No mixed type
No mixed type
No mixed type
No mixed type
No mixed type
No mixed type
No mixed type


In [13]:
# check missing values
cust_dropcol.isnull().sum()

user_id         0
gender          0
state           0
age             0
n_dependants    0
fam_status      0
income          0
dtype: int64

In [14]:
# check duplicates
cust_dropcol[cust_dropcol.duplicated()]

Empty DataFrame
Columns: [user_id, gender, state, age, n_dependants, fam_status, income]
Index: []

cust_dropcol dataframe is clean and ready for next step.

In [15]:
# export wrangled and clean custom dataset
cust_dropcol.to_csv(r'/Users/r1121/Desktop/07-2023 Instacart Basket Analysis/02 Data/Prepared Data/customers_wrangled_clean.csv')

# combine custom dataset with orders_products_merged_inner_4.9 dataset

In [16]:
# import op_merged.pkl data
op_merged = pd.read_pickle(r'/Users/r1121/Desktop/07-2023 Instacart Basket Analysis/02 Data/Prepared Data/orders_products_merged_inner_4.9.pkl')

In [17]:
op_merged.head(20)

order_id  user_id  order_number  order_dow  order_hour_of_day  \
0    2539329        1             1          2                  8   
1    2398795        1             2          3                  7   
2     473747        1             3          3                 12   
3    2254736        1             4          4                  7   
4     431534        1             5          4                 15   
5    3367565        1             6          2                  7   
6     550135        1             7          1                  9   
7    3108588        1             8          1                 14   
8    2295261        1             9          1                 16   
9    2550362        1            10          4                  8   
10   2968173       15            15          1                  9   
11   1870022       15            17          2                 16   
12   1911383       15            18          2                 11   
13   2715276       15            21          1                  9   
14    487368       15            22          1                 10   
15   2293453       19             2          5                 14   
16   1973799       19             5          6                 12   
17    532817       19             7          4                 17   
18   1573906       21            10          3                 10   
19   1593000       31            10          3                  8   

    days_since_prior_order  product_id  add_to_cart_order  reordered  \
0                      NaN         196                  1          0   
1                     15.0         196                  1          1   
2                     21.0         196                  1          1   
3                     29.0         196                  1          1   
4                     28.0         196                  1          1   
5                     19.0         196                  1          1   
6                     20.0         196                  1          1   
7                     14.0         196                  2          1   
8                      0.0         196                  4          1   
9                     30.0         196                  1          1   
10                     7.0         196                  2          0   
11                     8.0         196                  6          1   
12                     7.0         196                  1          1   
13                     7.0         196                  1          1   
14                    14.0         196                  1          1   
15                     6.0         196                  3          0   
16                     8.0         196                 15          1   
17                     6.0         196                  1          1   
18                     6.0         196                  2          0   
19                     7.0         196                 17          0   

   product_name  ...    price_range_loc     busiest_day    busiest_days  \
0          Soda  ...  Mid_range rpoduct  Regularly busy  Regularly busy   
1          Soda  ...  Mid_range rpoduct  Regularly busy      Least busy   
2          Soda  ...  Mid_range rpoduct  Regularly busy      Least busy   
3          Soda  ...  Mid_range rpoduct      Least busy      Least busy   
4          Soda  ...  Mid_range rpoduct      Least busy      Least busy   
5          Soda  ...  Mid_range rpoduct  Regularly busy  Regularly busy   
6          Soda  ...  Mid_range rpoduct  Regularly busy    Busiest days   
7          Soda  ...  Mid_range rpoduct  Regularly busy    Busiest days   
8          Soda  ...  Mid_range rpoduct  Regularly busy    Busiest days   
9          Soda  ...  Mid_range rpoduct      Least busy      Least busy   
10         Soda  ...  Mid_range rpoduct  Regularly busy    Busiest days   
11         Soda  ...  Mid_range rpoduct  Regularly busy  Regularly busy   
12         Soda  ...  Mid_range rpoduct  Regularly busy  Regularly

In [18]:
op_merged.columns

Index(['order_id', 'user_id', 'order_number', 'order_dow', 'order_hour_of_day',
       'days_since_prior_order', 'product_id', 'add_to_cart_order',
       'reordered', 'product_name', 'aisle_id', 'department_id', 'prices',
       'price_range_loc', 'busiest_day', 'busiest_days',
       'busiest_period_of_day', 'max_number', 'loyalty_flag',
       'user_purchased_mean', 'spender_flag', 'median_days_spo',
       'order_frq_flag'],
      dtype='object')

In [19]:
op_merged.shape

(32404859, 23)

In [20]:
# pick columns that related to analysis from op_merged data
op_merged_pick = op_merged[['order_id', 'user_id', 'product_id', 'product_name', 'prices', 'busiest_days',
                            'loyalty_flag', 'user_purchased_mean', 'spender_flag', 'median_days_spo',
                            'order_frq_flag']]

In [21]:
op_merged_pick.head(10)

order_id  user_id  product_id product_name  prices    busiest_days  \
0   2539329        1         196         Soda     9.0  Regularly busy   
1   2398795        1         196         Soda     9.0      Least busy   
2    473747        1         196         Soda     9.0      Least busy   
3   2254736        1         196         Soda     9.0      Least busy   
4    431534        1         196         Soda     9.0      Least busy   
5   3367565        1         196         Soda     9.0  Regularly busy   
6    550135        1         196         Soda     9.0    Busiest days   
7   3108588        1         196         Soda     9.0    Busiest days   
8   2295261        1         196         Soda     9.0    Busiest days   
9   2550362        1         196         Soda     9.0      Least busy   

   loyalty_flag  user_purchased_mean spender_flag  median_days_spo  \
0  New customer             6.367797  Low spender             20.5   
1  New customer             6.367797  Low spender             20.5   
2  New customer             6.367797  Low spender             20.5   
3  New customer             6.367797  Low spender             20.5   
4  New customer             6.367797  Low spender             20.5   
5  New customer             6.367797  Low spender             20.5   
6  New customer             6.367797  Low spender             20.5   
7  New customer             6.367797  Low spender             20.5   
8  New customer             6.367797  Low spender             20.5   
9  New customer             6.367797  Low spender             20.5   

          order_frq_flag  
0  Non-frequent customer  
1  Non-frequent customer  
2  Non-frequent customer  
3  Non-frequent customer  
4  Non-frequent customer  
5  Non-frequent customer  
6  Non-frequent customer  
7  Non-frequent customer  
8  Non-frequent customer  
9  Non-frequent customer

In [22]:
op_merged_pick.shape

(32404859, 11)

In [23]:
# merge two dataframes
op_cust_merged = op_merged_pick.merge(cust_dropcol, on = ['user_id'], indicator = True, how = 'left')

In [24]:
# check the output
op_cust_merged.head(10)

order_id  user_id  product_id product_name  prices    busiest_days  \
0   2539329        1         196         Soda     9.0  Regularly busy   
1   2398795        1         196         Soda     9.0      Least busy   
2    473747        1         196         Soda     9.0      Least busy   
3   2254736        1         196         Soda     9.0      Least busy   
4    431534        1         196         Soda     9.0      Least busy   
5   3367565        1         196         Soda     9.0  Regularly busy   
6    550135        1         196         Soda     9.0    Busiest days   
7   3108588        1         196         Soda     9.0    Busiest days   
8   2295261        1         196         Soda     9.0    Busiest days   
9   2550362        1         196         Soda     9.0      Least busy   

   loyalty_flag  user_purchased_mean spender_flag  median_days_spo  \
0  New customer             6.367797  Low spender             20.5   
1  New customer             6.367797  Low spender             20.5   
2  New customer             6.367797  Low spender             20.5   
3  New customer             6.367797  Low spender             20.5   
4  New customer             6.367797  Low spender             20.5   
5  New customer             6.367797  Low spender             20.5   
6  New customer             6.367797  Low spender             20.5   
7  New customer             6.367797  Low spender             20.5   
8  New customer             6.367797  Low spender             20.5   
9  New customer             6.367797  Low spender             20.5   

          order_frq_flag  gender    state  age  n_dependants fam_status  \
0  Non-frequent customer  Female  Alabama   31             3    married   
1  Non-frequent customer  Female  Alabama   31             3    married   
2  Non-frequent customer  Female  Alabama   31             3    married   
3  Non-frequent customer  Female  Alabama   31             3    married   
4  Non-frequent customer  Female  Alabama   31             3    married   
5  Non-frequent customer  Female  Alabama   31             3    married   
6  Non-frequent customer  Female  Alabama   31             3    married   
7  Non-frequent customer  Female  Alabama   31             3    married   
8  Non-frequent customer  Female  Alabama   31             3    married   
9  Non-frequent customer  Female  Alabama   31             3    married   

   income _merge  
0   40423   both  
1   40423   both  
2   40423   both  
3   40423   both  
4   40423   both  
5   40423   both  
6   40423   both  
7   40423   both  
8   40423   both  
9   40423   both

In [25]:
op_cust_merged['_merge'].value_counts()

both          32404859
left_only            0
right_only           0
Name: _merge, dtype: int64

This means that all user_id are matched in these two dataframes.

In [26]:
# export the new combined dataframe to prepared data folder.
op_cust_merged.to_pickle(r'/Users/r1121/Desktop/07-2023 Instacart Basket Analysis/02 Data/Prepared Data/orders_products_customers_merged.pkl')

In [27]:
op_cust_merged.shape

(32404859, 18)